# Face Recognition Using mlrun with OpenCV And PyTorch
 A complete pipeline of data processing, model training and serving function deployment.

### Declare global variables

In [1]:
ARTIFACTS_PATH = '/User/demos/demos/faces/artifacts/'
CODE_PATH = '/User/demos/demos/faces/code/'

In [2]:
use_gpu = False

### Import and define mlrun functions for the pipeline 

In [3]:
# nuclio: ignore
from mlrun import new_function, code_to_function, NewTask, mount_v3io
import kfp
from kfp import dsl

In [4]:
encode_fn = code_to_function('encode-images', image='yjbds/mlrun-horovod-gpu:0.4.5', kind='job', filename=CODE_PATH + 'encode.py')
encode_fn.export(CODE_PATH + 'encode.yaml')
encode_fn.apply(mount_v3io())

[mlrun] 2020-05-13 14:01:00,293 function spec saved to path: /User/demos/demos/faces/code/encode.yaml


In [5]:
if use_gpu:
    train_fn = new_function(name='train', image='yjbds/mlrun-horovod-gpu:0.4.5', kind='mpijob', command=CODE_PATH + 'horovod_train.py')
    train_fn.gpus(1)

else:
    train_fn = new_function(name='train', image='yjbds/mlrun-horovod-gpu:0.4.5', kind='job', command=CODE_PATH + 'train.py') 

train_fn.export(CODE_PATH + 'train.yaml')
train_fn.spec.replicas = 2
train_fn.apply(mount_v3io())

[mlrun] 2020-05-13 14:01:00,827 function spec saved to path: /User/demos/demos/faces/code/train.yaml


In [6]:
model_serving_function = code_to_function(
    name='recognize-faces', 
    filename='./nuclio-face-prediction.ipynb',
    kind='nuclio')

model_serving_function.apply(mount_v3io())

In [7]:
api_serving_function = code_to_function(
    name='video-api-server', 
    filename='./nuclio-api-serving.ipynb',
    kind='nuclio')

api_serving_function.apply(mount_v3io())

In [8]:
from mlrun import mlconf
mlconf.dbpath = 'http://mlrun-api:8080'

### Create pipeline

In [12]:
@dsl.pipeline(
    name='face recognition pipeline',
    description='Creates and deploys a face recognition model'
)
def face_recognition_pipeline(with_cuda=False, with_horovod=False):
    
    encode = encode_fn.as_step(name='encode-images', handler='encode_images', artifact_path=ARTIFACTS_PATH, outputs=['idx2name', 'encodings_path'],
                       inputs={'cuda': with_cuda})
    
    train = train_fn.as_step(name='train', artifact_path=ARTIFACTS_PATH, outputs=['model'], 
                               inputs={'processed_data': encode.outputs['encodings_path']})
    
    deploy_model = model_serving_function.deploy_step(project='default', models={'face_rec_v1': train.outputs['model']})
    
    deploy_api = api_serving_function.deploy_step(project='default').after(deploy_model)
    

In [13]:
client = kfp.Client(namespace='default-tenant')

In [14]:
#For debug purposes compile pipeline code
kfp.compiler.Compiler().compile(face_recognition_pipeline, 'face_rec.yaml')

### Run pipeline

In [15]:
arguments = {}
run_result = client.create_run_from_pipeline_func(face_recognition_pipeline, arguments=arguments, run_name='face_rec_1', experiment_name='face_rec')